In [1]:
# classifiy foes and enemies using graph theory
import networkx as nx
import pandas as pd
samples=pd.read_csv('islander sample people scores.csv')
friends=pd.read_csv('friends.csv')
foes=pd.read_csv('foes.csv')
samples.drop(samples[(samples['person id']=='91dade1d_b0c4091a')|(samples['person id']=='45ecc3c3_559a1b05')|(samples['person id']=='ce45ca9_3fa2bac9')].index, axis=0, inplace=True)
samples=samples.reset_index(drop=True)

In [2]:
if 'g' in globals(): g.clear()
g = nx.Graph()
## create a network of sample Persons
for i in range(len(samples)):
    g.add_node(samples['person id'][i],score=samples['score'][i])

In [3]:
# add friendly edges
for i in range(len(friends)):
    p1=friends['person1'][i]
    if g.has_node(p1)==False:
        g.add_node(p1,score=0)
    p2=friends['person2'][i]
    if g.has_node(p2)==False:
        g.add_node(p2,score=0)
    if (g.has_edge(p1,p2))==False:
        g.add_edge(p1, p2, weight=1, probability=0.91)

In [4]:
#add foe type edges
for i in range(len(foes)):
    p1=foes['person1'][i]
    if g.has_node(p1)==False:
        g.add_node(p1,score=0)
    p2=foes['person2'][i]
    if g.has_node(p2)==False:
        g.add_node(p2,score=0)
    if (g.has_edge(p1,p2))==False:
        g.add_edge(p1, p2, weight=-1, probability=0.99)

In [5]:
#correction using fist condition
for node in g.nodes():
    if any(samples['person id']==node)==False: continue
    neighbors=g[node]
    for neighbor in neighbors:
        if (g.node[node]['score']>0)&(g.node[neighbor]['score']<0)&((g.edges[(node,neighbor)]['weight'])>0):
            g.edges[(node,neighbor)]['weight']=-1

In [6]:
#correction using second condition
for node in g.nodes():
    neighbors=g[node]
    for neighbor in neighbors:
        if ((g.edges[(node,neighbor)]['weight'])==1):
            n_neighbors=g[neighbor]
            for n_neighbor in n_neighbors:
                if g.has_edge(node,n_neighbor):
                    if (g.edges[(node,n_neighbor)]['weight'])==-1:
                        g.edges[(node,n_neighbor)]['weight']=1

In [7]:
#correction using third condition
for node in g.nodes():
    neighbors=g[node]
    for neighbor in neighbors:
        if ((g.edges[(node,neighbor)]['weight'])==-1):
            n_neighbors=g[neighbor]
            for n_neighbor in n_neighbors:
                if g.has_edge(node,n_neighbor):
                    if (g.edges[(node,n_neighbor)]['weight'])==1:
                        g.edges[(node,n_neighbor)]['weight']=-1

In [8]:
# score simulation
# repeat for 30 time periods
for i in range(30):
    error=0
    ## iterate through all nodes in the network and tell them to make a step
    for node in g.nodes():
        if any(samples['person id']==node): continue
        neighbors=g[node]
        w=1/float((len(neighbors)))
        s=0
        for neighbor in neighbors:
            if (g.node[node]['score']>0)&(g.node[neighbor]['score']<0)&((g.edges[(node,neighbor)]['weight'])==1):
                g.edges[(node,neighbor)]['weight']=-1
            if (g.node[node]['score']<0)&(g.node[neighbor]['score']>0)&((g.edges[(node,neighbor)]['weight'])==1):
                g.edges[(node,neighbor)]['weight']=-1
            edge_weight=g.edges[(node,neighbor)]['weight']*g.edges[(node,neighbor)]['probability']
            s+=w*g.node[neighbor]['score']*edge_weight
        # update my beliefs = initial belief plus sum of all influences
        error+=abs(g.node[node]['score']-s)
        g.node[node]['score']=s
    print('iteration no. = ',i,' , absolute error= ',error)

iteration no. =  0  , absolute error=  9111.257565791311
iteration no. =  1  , absolute error=  11876.699068564154
iteration no. =  2  , absolute error=  8982.166453074786
iteration no. =  3  , absolute error=  6307.542682309307
iteration no. =  4  , absolute error=  4393.040520922266
iteration no. =  5  , absolute error=  3064.490630887777
iteration no. =  6  , absolute error=  2141.980926609277
iteration no. =  7  , absolute error=  1498.4931540796733
iteration no. =  8  , absolute error=  1050.1768363790004
iteration no. =  9  , absolute error=  735.4494944969127
iteration no. =  10  , absolute error=  515.2497285093522
iteration no. =  11  , absolute error=  361.0629209234928
iteration no. =  12  , absolute error=  253.02352574019773
iteration no. =  13  , absolute error=  177.3080372891834
iteration no. =  14  , absolute error=  124.24379271624638
iteration no. =  15  , absolute error=  87.0554279398409
iteration no. =  16  , absolute error=  60.99468202638582
iteration no. =  17 

In [19]:
#use initial friendships to find unlikely cases
friends_reported=pd.read_csv('friends.csv')

In [22]:
friends_reported=pd.read_csv('friends.csv')
friends_reported['diff']=0
for i in range(len(friends_reported)):
    p1=friends_reported['person1'][i]
    p2=friends_reported['person2'][i]
    friends_reported.loc[i,'diff']=abs(g.node[p1]['score']-g.node[p2]['score'])

In [23]:
#answer to part(1)
friends_reported.sort_values(by=['diff'], ascending=False).head(8)

,person1,person2,diff
210753,70af45c5_3ff6612,2f878b2c_55a1556d,7.0
329184,6c9a6b64_2f633d36,70af45c5_3ff6612,7.0
156227,4994d25f_fb1355ee,a8a32a9f_ae184492,7.0
254223,d2c95676_56a7db59,a64c47ea_dbafbd1d,7.0
138126,70af45c5_3ff6612,7e1b8296_b7dcbaca,7.0
246007,83569979_6e3eedcc,1632bec2_52d70733,7.0
154890,73fe0531_893dcdb6,5cd4c2df_c496ada1,7.0
390991,d387bc1c_568ccd2d,376bae6f_314cbe19,7.0


In [26]:
#answer to part(2)
dic={}
for node in g.nodes():
    if g.node[node]['score']>0:
        dic[node]=(g.node[node]['score'],'good')
    else:
        dic[node]=(g.node[node]['score'],'bad')

In [27]:
dic

{'1122b7d5_ed317db7': (-2, 'bad'),
 'b0177400_be9028c7': (-2, 'bad'),
 '77ac4e2a_77109d8c': (-1, 'bad'),
 '49ca4e7d_2eb499f': (2, 'good'),
 '9e4a7320_9ebed277': (-2, 'bad'),
 '52936684_6bd2c2f3': (3, 'good'),
 'bb891fa8_b2f18316': (2, 'good'),
 '676be448_ab12f528': (2, 'good'),
 'fa71973b_dfeba98c': (2, 'good'),
 '3166f635_244cda27': (2, 'good'),
 'a043640d_d3e5ce20': (-2, 'bad'),
 '7fa6c1d3_4ef6525f': (3, 'good'),
 '9e858e0f_c50fe1cb': (2, 'good'),
 'd1b5f6f0_4731e34e': (-2, 'bad'),
 '3af85c6e_f4748112': (1, 'good'),
 'fc9f1cf6_5bb90d16': (1, 'good'),
 '634c20b3_6d1a30df': (4, 'good'),
 'a0ba99cd_cbb1d5a': (-3, 'bad'),
 'd8c7f894_30f97f24': (-2, 'bad'),
 '6463f790_f24d25a3': (-2, 'bad'),
 '965a49fe_fa4b2d5': (2, 'good'),
 '8390f47d_1ecbe6e': (-2, 'bad'),
 '33339867_fa9c7bae': (-2, 'bad'),
 '5ab6fb0e_55091c4d': (2, 'good'),
 '354f95a6_f3a3fcda': (3, 'good'),
 '3808972d_6dccb11d': (-2, 'bad'),
 'ef0a6174_164d6764': (-2, 'bad'),
 '4c427af6_e3aee3b5': (-2, 'bad'),
 '139a1143_2f017e61': (-